# GluonCV Image Classification - AWS Marketplace Model Package

This tutorial walks you through the fundamental usage of GluonCV image classification models in AWS Marketplace.


## Essential Sagemaker imports

In [1]:
import sagemaker as sage
import os
import boto3
from sagemaker import get_execution_role
from sagemaker import transformer
from sagemaker import model

role = get_execution_role()

##  Create the session

In [2]:
sess = sage.Session()

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
region

'us-west-2'

## Subscribe to the model

Before you can deploy the model, your account needs to be subscribed to it.

- Open the Model Package listing page and search for GluonCV image classification models .
- Read the product overview section and Highlights section to understand the value proposition of our model package.
- View usage information and then additional resources sections. 
- Click on Continue to Subscribe button & then Click on "Accept Offer" button on the next page.
- Next, Continue to Configuration button becomes activated and when you click on the button, you will see that a Product Arn will appear.
- You have successfully subscribed to our model package. You can find it in your SageMaker console. Keep record of the model package arn you have subscribed to.
- Close the page and continue executing this notebook.

In [3]:
modelpackage_arn = 'arn:aws:sagemaker:us-west-2:985964311364:model-package/gluoncv-resnet50-v1b-1594338795'

## Create Model

In [4]:
from sagemaker import ModelPackage

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='image/jpeg')

# Create a deployable model
model = ModelPackage(role=role,
                     model_package_arn = modelpackage_arn,
                     sagemaker_session = sess,
                     predictor_cls = predict_wrapper)
endpoint_name = 'gluoncv-resnet50'

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


## Create live Inference Endpoint

Now that we have the model created, we can launch the live inference endpoint, this may take a while in the background.

In [5]:
predictor = model.deploy(1, 'ml.m4.xlarge', endpoint_name=endpoint_name)

-------------!

## Let's grab some demo images for inference

In [6]:
!wget https://github.com/zhreshold/gluoncv-sagemaker-serving/raw/master/data/transform/cat1.jpg -O cat.jpg

--2020-08-21 23:07:57--  https://github.com/zhreshold/gluoncv-sagemaker-serving/raw/master/data/transform/cat1.jpg
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zhreshold/gluoncv-sagemaker-serving/master/data/transform/cat1.jpg [following]
--2020-08-21 23:07:57--  https://raw.githubusercontent.com/zhreshold/gluoncv-sagemaker-serving/master/data/transform/cat1.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53081 (52K) [image/jpeg]
Saving to: ‘cat.jpg’

cat.jpg             100%[===================>]  51.84K  --.-KB/s    in 0.01s   

2020-08-21 23:07:57 (4.02 MB/s) - ‘cat.jpg’ saved [53081/53081]



We can perform a typical image classification task with the live endpoint by posting the raw image data

In [7]:
file_name = 'cat.jpg'
# Open the file and read the image into a bytearray.
with open(file_name, "rb") as image:
  b = bytearray(image.read())

# Perform a prediction
result = predictor.predict(b).decode('utf-8')

with open("output.txt", "w") as text_file:
    text_file.write(result)
    
# View the prediction
print(result)

{"prediction": [{"class": "tiger cat", "prob": "0.4041753"}, {"class": "lynx", "prob": "0.28042606"}, {"class": "tabby", "prob": "0.037456095"}, {"class": "Egyptian cat", "prob": "0.029903224"}, {"class": "Siamese cat", "prob": "0.013234771"}]}


The image classifier can also be used as image feature extractor at different layers

In [26]:
import json
# retrieve the 'fc'- feature of the fully connected layer
result = predictor.predict(b, initial_args={'CustomAttributes': '{"feature": "fc"}'}).decode('utf-8')

# load the feature, slice the prediction [0] as the first output
feature = json.loads(result)['feature'][0]
print('length of feature:', len(feature))
print('peek for features', feature[:100])

length of feature: 1000
peek for features [0.19229421019554138, 0.22757908701896667, -0.4809921979904175, 0.4378524720668793, 0.5704623460769653, 0.39407360553741455, -0.9311790466308594, -0.43465763330459595, -1.1562561988830566, -0.6035076379776001, -0.03237644582986832, 0.795693039894104, 0.753682017326355, -0.5762457251548767, 0.42405959963798523, 0.47193852066993713, -0.4915999174118042, -0.7994516491889954, 0.6871213912963867, -0.15457800030708313, -0.2068454921245575, -0.8789792060852051, 0.4469362199306488, 1.107102394104004, -0.4018600285053253, 0.7917894721031189, -0.6949979066848755, 0.15867739915847778, 0.4145841598510742, -0.42396003007888794, 0.3196070194244385, 0.33665621280670166, 0.8471510410308838, 0.9467760324478149, 1.7589367628097534, -0.986995279788971, -0.29512137174606323, 0.572476863861084, 0.7496881484985352, -0.04598068445920944, -0.309583842754364, 0.7713222503662109, 1.3152652978897095, 0.1485675573348999, 0.27398067712783813, 0.20475608110427856, -0.380604

##  Delete the Endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. you can terminate the same to avoid being charged.

In [27]:
predictor = sage.RealTimePredictor(endpoint_name, sess, content_type='image/jpeg')
predictor.delete_endpoint(delete_endpoint_config=True)